In [1]:
from ase.atoms import Atoms
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.constraints import UnitCellFilter
from ase.eos import calculate_eos, plot
from ase.data import reference_states, atomic_numbers
from ase.optimize import LBFGS
from ase.units import kJ

In [2]:
from mace.calculators import mace_mp

In [3]:
from langchain.agents import AgentExecutor
from langchain.agents import tool
from langchain_core.pydantic_v1 import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

In [4]:
from typing import List
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import ipywidgets as widgets

In [6]:
class AtomsDict(BaseModel):
    numbers: List[int]
    positions: List[List[float]]
    cell: List[List[float]]
    pbc: List[bool]

In [7]:
def get_calculator(calculator_str: str = "emt"):
    if calculator_str == "emt":
        return EMT()
    elif calculator_str == "mace":
        return mace_mp(
            model="medium",
            dispersion=False,
            default_dtype="float32",
            device='cpu'
        )

In [8]:
@tool 
def get_equilibirum_lattice(chemical_symbol: str, calculator_str: str) -> AtomsDict:
    """Returns equilibrium atoms dictionary for a given chemical symbol and a selected model specified by the calculator string"""
    atoms = bulk(name=chemical_symbol)
    atoms.calc = get_calculator(calculator_str=calculator_str)
    ase_optimizer_obj = LBFGS(UnitCellFilter(atoms))
    ase_optimizer_obj.run(fmax=0.000001)
    return AtomsDict(**{k: v.tolist() for k, v in atoms.todict().items()})

In [9]:
@tool
def plot_equation_of_state(atom_dict: AtomsDict, calculator_str: str) -> str:
    """Returns plot of equation of state of chemical symbol for a given atoms dictionary and a selected model specified by the calculator string"""
    atoms = Atoms(**atom_dict.dict())
    atoms.calc = get_calculator(calculator_str=calculator_str)
    eos = calculate_eos(atoms)
    plotdata = eos.getplotdata()
    return plot(*plotdata)

In [10]:
@tool
def get_bulk_modulus(atom_dict: AtomsDict, calculator_str: str) -> str:
    """Returns the bulk modulus of chemcial symbol for a given atoms dictionary and a selected model specified by the calculator string in GPa"""
    atoms = atoms = Atoms(**atom_dict.dict())
    atoms.calc = get_calculator(calculator_str=calculator_str)
    eos = calculate_eos(atoms)
    v, e, B = eos.fit()
    return B / kJ * 1.0e24

In [11]:
@tool
def get_equilibrium_volume(atom_dict: AtomsDict, calculator_str: str) -> str:
    """Returns the equilibrium volume of chemcial symbol for a given atoms dictionary and a selected model specified by the calculator string in Angstrom^3"""
    atoms = atoms = Atoms(**atom_dict.dict())
    atoms.calc = get_calculator(calculator_str=calculator_str)
    eos = calculate_eos(atoms)
    v, e, B = eos.fit()
    return v

# Simulation LLM
The first large language model for calculating material properties. Example queries: 
* "Calculate the bulk modulus for aluminium with the mace model"
* "Calculate the equilibrium volume of gold"

In [12]:
key_widget = widgets.Text(
    value='sk-********************',
    placeholder='',
    description='OpenAI Token:',
    disabled=False,
    layout=widgets.Layout(width='400px'),
    style = {'description_width': '100px'} 
)

In [13]:
query_widget = widgets.Textarea(
    value='Calculate the equilibrium volume of gold',
    placeholder='',
    description='Next query:',
    disabled=False,
    layout=widgets.Layout(width='400px'),
    style = {'description_width': '100px'}     
)

In [14]:
def get_extension(OPENAI_API_KEY):
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)
    tools = [get_equilibrium_volume, get_equilibirum_lattice, plot_equation_of_state, get_bulk_modulus]
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are very powerful assistant, but don't know current events. To calculate with emt use the calculator string emt and to calculate with mace use the calculator string mace. For each query vailidate that it contains a chemical element and a calculator string and otherwise use Alumninum as the default chemical element and emt as the default calculator string.",
            ),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    llm_with_tools = llm.bind_tools(tools)
    agent = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
    )
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

In [52]:
def run_refresh(*ignore):
    agent_executor = get_extension(OPENAI_API_KEY=key_widget.value)
    output = list(agent_executor.stream({"input": query_widget.value}))
    with out:
        display(output[-1]['output'])

In [53]:
out = widgets.Output()

In [54]:
button = widgets.Button(description ="Submit")
button.on_click(run_refresh)

In [55]:
display(query_widget, key_widget, button, out)

Text(value='Calculate the equilibrium volume of aluminium', description='Next query:', placeholder='')

Text(value='sk-****************', description='OpenAI Token:', placeholder='')

Button(description='Submit', style=ButtonStyle())

Output()